# Hybrid model - soft voting


## Python imports

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import pickle
import tensorflow as tf

2025-06-14 11:47:01.465101: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-14 11:47:01.509246: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-14 11:47:01.509905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-14 11:47:02.184571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Parameters

In [2]:
BRSSI_MODEL_ID = 'RandomForest'
VISION_MODEL_ID = 'MobileNetV1'
TEST_SETS = { 'TS', 'PW', 'RW'}
LABELS = [
    "AH", "AT_CA", "AT_CH", "AT_I1", "AT_I2", "AT_M", "AT_O1",
    "AT_O2", "AT_S", "CN", "DC", "DF", "DG", "ES",
    "GL", "HA", "SA", "SN", "SS", "TMA", "TS"
]
BRSSI_MODEL_WEIGHT = 0.5
VISION_MODEL_WEIGHT = 0.5

## Load BRSSI and vision models 

In [3]:
BRSSI_MODEL = pickle.load(open('models/brssi/' + BRSSI_MODEL_ID + '.pkl','rb'))
VISION_MODEL = tf.keras.models.load_model('models/vision/' + VISION_MODEL_ID )

## Procedure to compute soft voting results for a test set

In [4]:
def calc_soft_voting_results(test_set_id):
    # Load BRSSI and vision datasets
    brssi_data = pd.read_csv('results/brssi/' + BRSSI_MODEL_ID + '_' + test_set_id + '.tsv', sep='\t')
    brssi_data.sort_values(['label','source','time'],inplace=True)
    vision_data = pd.read_csv('results/vision/' + VISION_MODEL_ID + '_' + test_set_id + '.tsv', sep='\t')
    vision_data.sort_values(['label','source','time'],inplace=True)
    # align data
    brssi_data = brssi_data.merge(vision_data[['label','time','source']])
    vision_data = vision_data.merge(brssi_data[['label','time','source']])
    # get BRSSI model results
    brssi_outputs = brssi_data[LABELS]
    # get vision model results
    vision_outputs = vision_data[LABELS]
    # compute soft voting outputs
    fusion_scores = BRSSI_MODEL_WEIGHT * brssi_outputs + VISION_MODEL_WEIGHT * vision_outputs
    #top_cat_index = [ fs.argsort()[::-1][0] for fs in fusion_scores ]
    predictions = fusion_scores.idxmax(axis='columns')
    print(predictions)
    results = brssi_data[['label','time','source']].copy()
    results['prediction'] = predictions
    for l in LABELS:
        results[l] = fusion_scores[l]
    results.to_csv('results/hybrid/SoftVoting_'+ test_set_id + '.tsv', header=True, index=False, sep='\t')
    tmp = pd.DataFrame()
    tmp['correct_prediction'] = (results['label'] == results['prediction']).astype(float) 
    acc = tmp['correct_prediction'].mean()
    print(test_set_id, acc)
    return results

In [5]:
calc_soft_voting_results('TS')

0      AH
1      AH
2      AH
3      AH
4      AH
       ..
936    TS
937    DG
938    TS
939    TS
940    TS
Length: 941, dtype: object
TS 0.9766206163655685


,label,time,source,prediction,AH,AT_CA,AT_CH,AT_I1,AT_I2,AT_M,...,DF,DG,ES,GL,HA,SA,SN,SS,TMA,TS
0,AH,8,Pixel,AH,0.898964,2.500137e-03,3.497898e-07,1.934008e-04,3.677962e-07,2.131400e-04,...,0.000472,9.758191e-03,2.526454e-04,2.841581e-06,1.781105e-06,0.042364,0.000229,9.335710e-07,1.135980e-04,0.025099
1,AH,12,Pixel,AH,0.585479,2.905041e-04,4.563791e-06,3.166023e-04,1.519484e-05,4.659339e-05,...,0.019542,1.809505e-02,1.314040e-01,8.183039e-05,1.615097e-02,0.040149,0.026206,1.737452e-03,2.503895e-02,0.029992
2,AH,17,Pixel,AH,0.960145,5.774826e-09,1.136566e-06,3.746181e-08,2.048399e-05,1.026337e-05,...,0.000422,1.371381e-07,3.055286e-03,1.584611e-05,1.522359e-05,0.012824,0.000635,2.582959e-03,2.481809e-03,0.017508
3,AH,19,Pixel,AH,0.860099,7.578371e-07,5.358547e-07,2.850346e-06,2.311017e-07,1.322142e-08,...,0.000422,2.847859e-03,1.479569e-04,1.370954e-07,6.700773e-05,0.015409,0.000005,1.006349e-06,1.827652e-05,0.117591
4,AH,23,Pixel,AH,0.644845,4.207632e-13,1.726639e-11,2.500000e-03,2.500023e-03,1.359438e-11,...,0.002500,1.805017e-16,2.500218e-03,9.959936e-07,3.430800e-08,0.313357,0.002500,2.500002e-03,1.552487e-09,0.011796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,TS,78,Xiaomi,TS,0.001629,2.574101e-05,2.398589e-05,1.706678e-04,2.096021e-05,2.527604e-05,...,0.188331,3.093817e-02,3.311984e-03,2.137117e-08,1.533662e-03,0.075483,0.000071,2.628293e-03,6.530781e-03,0.669374
937,TS,88,Xiaomi,DG,0.210843,1.713715e-04,6.442221e-05,1.305882e-02,2.568148e-05,2.857027e-04,...,0.018654,3.542859e-01,3.119788e-03,1.387719e-07,7.492103e-03,0.025231,0.002195,1.307582e-06,2.003866e-04,0.319822
938,TS,95,Xiaomi,TS,0.050025,1.078146e-04,5.076968e-03,7.362276e-05,2.133363e-03,6.731780e-04,...,0.131096,1.882061e-02,9.561904e-03,5.907276e-07,1.322082e-03,0.006050,0.000053,1.267973e-05,6.248811e-03,0.525307
939,TS,101,Xiaomi,TS,0.025677,5.723930e-06,2.501176e-03,3.361632e-04,2.505234e-03,8.787173e-06,...,0.155609,2.312614e-01,1.062320e-03,2.348654e-09,5.700074e-05,0.005594,0.000049,3.951652e-07,2.603235e-03,0.549049


In [6]:
calc_soft_voting_results('PW')

0      TS
1      AH
2      AH
3      AH
4      AH
       ..
358    TS
359    TS
360    TS
361    CN
362    TS
Length: 363, dtype: object
PW 0.8181818181818182


,label,time,source,prediction,AH,AT_CA,AT_CH,AT_I1,AT_I2,AT_M,...,DF,DG,ES,GL,HA,SA,SN,SS,TMA,TS
0,AH,92,Pixel,TS,0.210500,5.920237e-06,1.194791e-05,7.802424e-05,7.020085e-06,4.703154e-07,...,3.666811e-03,1.275606e-02,2.166847e-05,4.979001e-08,4.230071e-06,0.011527,2.815273e-06,2.326071e-07,5.261352e-04,0.760484
1,AH,93,Pixel,AH,0.952481,2.970059e-10,4.747464e-09,3.667097e-11,1.422812e-10,1.264320e-09,...,2.287604e-08,8.434025e-12,2.756234e-06,6.141456e-07,1.916476e-07,0.012500,2.935269e-08,5.959878e-08,2.304467e-07,0.035015
2,AH,94,Pixel,AH,0.992493,1.135669e-11,2.253152e-11,6.976355e-09,5.626569e-09,1.652988e-10,...,7.681005e-07,5.692602e-12,1.379725e-07,8.121695e-08,5.159922e-09,0.002500,2.122025e-10,4.031584e-10,1.712123e-07,0.005006
3,AH,95,Pixel,AH,0.962562,8.254130e-07,2.470860e-07,2.393430e-06,5.945334e-07,6.725190e-06,...,2.083073e-03,2.113946e-06,7.705231e-05,1.323019e-07,7.851364e-06,0.007550,9.954753e-06,1.025247e-07,1.380775e-05,0.025006
4,AH,96,Pixel,AH,0.758353,1.076826e-04,1.863334e-06,5.094707e-04,4.751765e-06,6.217011e-06,...,7.202524e-02,4.741019e-02,1.253182e-02,1.410040e-05,5.546207e-04,0.004154,7.902463e-03,6.924518e-05,1.873057e-02,0.037370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,TS,87,Pixel,TS,0.290178,4.766471e-07,9.626963e-06,4.515019e-06,4.488810e-07,1.947276e-06,...,8.717044e-03,2.235188e-05,2.066116e-05,2.030924e-07,1.404441e-06,0.002504,9.963331e-06,6.614201e-07,9.033115e-05,0.698367
359,TS,88,Pixel,TS,0.177581,2.201694e-08,9.405017e-09,6.085771e-06,2.501300e-03,6.265176e-07,...,2.222006e-04,1.005327e-05,6.330281e-07,3.248404e-08,7.165020e-08,0.002500,7.391755e-07,2.500001e-03,2.829562e-05,0.814638
360,TS,89,Pixel,TS,0.226152,1.044734e-04,8.983443e-06,1.682575e-03,7.614188e-04,1.182977e-03,...,1.376806e-02,3.105618e-02,3.982923e-03,1.107437e-06,5.578574e-04,0.025872,3.280573e-04,6.435926e-06,8.435190e-04,0.597921
361,TS,90,Pixel,CN,0.225001,8.871360e-06,8.476502e-06,5.265224e-04,6.844319e-05,2.664047e-04,...,4.741472e-03,1.337396e-01,9.222488e-04,1.455042e-08,2.995193e-05,0.004937,1.765028e-03,3.593237e-07,5.434544e-05,0.283780


In [7]:
calc_soft_voting_results('RW')

0      AH
1      AH
2      AH
3      AH
4      AH
       ..
441    TS
442    TS
443    TS
444    TS
445    TS
Length: 446, dtype: object
RW 0.9125560538116592


,label,time,source,prediction,AH,AT_CA,AT_CH,AT_I1,AT_I2,AT_M,...,DF,DG,ES,GL,HA,SA,SN,SS,TMA,TS
0,AH,112,Xiaomi,AH,0.679068,5.022805e-03,2.916831e-03,2.500640e-03,0.032503,0.000453,...,0.029757,0.007581,0.000060,6.411011e-08,2.504391e-03,0.100515,7.891276e-08,4.626304e-08,3.128378e-06,0.087994
1,AH,113,Xiaomi,AH,0.821968,4.292905e-06,5.047194e-03,4.321064e-07,0.005095,0.000018,...,0.007766,0.005428,0.000019,2.477591e-08,1.353606e-06,0.070867,2.071337e-08,5.887814e-09,2.465528e-06,0.067620
2,AH,114,Xiaomi,AH,0.801129,5.008422e-03,9.630573e-03,1.546881e-06,0.023175,0.001213,...,0.012045,0.003316,0.000247,1.603443e-07,8.459417e-06,0.084836,6.685409e-07,2.312138e-08,3.676326e-06,0.008610
3,AH,115,Xiaomi,AH,0.723765,5.000318e-03,4.583335e-03,8.648076e-09,0.064507,0.003385,...,0.005864,0.000037,0.000021,1.967144e-08,4.169657e-08,0.138855,2.500319e-03,4.019690e-09,5.182885e-07,0.002545
4,AH,116,Xiaomi,AH,0.768336,1.089610e-05,5.062791e-03,5.795761e-07,0.012007,0.000555,...,0.003277,0.016205,0.002353,1.596717e-06,2.089969e-05,0.084085,1.782002e-03,8.160817e-07,2.237849e-04,0.015257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,TS,106,Xiaomi,TS,0.060559,6.642497e-07,1.500054e-02,7.520566e-03,0.017505,0.002501,...,0.009185,0.093124,0.000013,2.911931e-08,3.679072e-06,0.011553,3.818531e-05,4.323575e-07,1.941890e-05,0.745437
442,TS,107,Xiaomi,TS,0.111280,1.395736e-07,1.094300e-02,5.434107e-03,0.017510,0.005681,...,0.006206,0.022501,0.000002,7.793306e-06,1.150331e-06,0.025003,4.091413e-06,2.500001e-03,1.357417e-04,0.715311
443,TS,108,Xiaomi,TS,0.013314,1.528504e-07,2.914982e-07,2.510883e-03,0.000002,0.002500,...,0.000239,0.097500,0.000003,1.999758e-05,7.919437e-07,0.017506,2.387102e-06,1.697791e-08,2.501245e-03,0.831396
444,TS,109,Xiaomi,TS,0.045235,1.107263e-06,4.239014e-07,5.006637e-03,0.002501,0.002501,...,0.001017,0.107689,0.000230,3.196232e-07,3.785468e-07,0.005093,4.725153e-05,2.500232e-03,2.541103e-03,0.803128
